# RTpipeline on Google Colab - Part 2: CPU Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/rtpipeline/blob/main/rtpipeline_colab_part2_cpu.ipynb)

**💰 Cost Optimization:** This notebook runs on **CPU ONLY** (no GPU needed)!

## What This Part Does

✅ **Loads segmentations** from Part 1
✅ **DVH extraction** (dose-volume histogram metrics)
✅ **Radiomics features** (150+ texture/shape features)
✅ **Robustness testing** (optional - feature stability)
✅ **Aggregation and visualization**
✅ **Downloadable results**

## Prerequisites

- Completed Part 1 (GPU segmentation)
- Part 1 outputs saved to Google Drive
- **CPU runtime** (Runtime → Change runtime type → None/CPU)

---

**⚡ Quick Start:**
1. Run cells 1-3 (setup)
2. Mount Google Drive (cell 4)
3. **UPDATE CONFIGURATION** (cell 5) - Point to Part 1 output folder
4. Run remaining cells

## 1️⃣ Setup: Install Miniconda & System Dependencies

This takes ~2 minutes

In [ ]:
%%bash
# Install system dependencies
echo "=== Installing System Dependencies ==="
apt-get update -qq
apt-get install -y -qq dcm2niix pigz > /dev/null

# Install Miniconda
if [ ! -d "/content/miniconda" ]; then
    echo -e "\n=== Installing Miniconda ==="
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    bash /tmp/miniconda.sh -b -p /content/miniconda
    rm /tmp/miniconda.sh
    echo "✅ Miniconda installed"
else
    echo "✅ Miniconda already installed"
fi

# Initialize conda
export PATH="/content/miniconda/bin:$PATH"
eval "$(/content/miniconda/bin/conda shell.bash hook)"
conda init bash

echo -e "\n✅ Setup complete!"

## 2️⃣ Clone RTpipeline Repository

In [ ]:
%%bash
if [ ! -d "/content/rtpipeline" ]; then
    echo "Cloning rtpipeline repository..."
    git clone -q https://github.com/kstawiski/rtpipeline.git /content/rtpipeline
    echo "✅ Repository cloned"
else
    echo "✅ Repository already exists"
    cd /content/rtpipeline
    git pull origin main
    echo "Repository updated"
fi

## 3️⃣ Create Conda Environments

This creates two environments (~5-10 minutes, only once per session):
- **rtpipeline**: For DVH extraction
- **rtpipeline-radiomics**: For PyRadiomics

In [ ]:
%%bash
export PATH="/content/miniconda/bin:$PATH"
eval "$(/content/miniconda/bin/conda shell.bash hook)"

# Accept ToS
echo "=== Accepting Anaconda Terms of Service ==="
conda config --set channel_priority flexible
if ! conda tos accept --channel defaults 2>&1; then
    echo "⚠️ ToS acceptance failed or already accepted"
fi
echo "✅ ToS accepted"

cd /content/rtpipeline

# Create rtpipeline environment
if conda env list | grep -q "^rtpipeline "; then
    echo "✅ Environment 'rtpipeline' exists"
else
    echo "Creating 'rtpipeline' environment..."
    conda env create -f envs/rtpipeline.yaml -q
    echo "✅ Created"
fi

# Create rtpipeline-radiomics environment
if conda env list | grep -q "^rtpipeline-radiomics "; then
    echo "✅ Environment 'rtpipeline-radiomics' exists"
else
    echo "Creating 'rtpipeline-radiomics' environment..."
    conda env create -f envs/rtpipeline-radiomics.yaml -q
    echo "✅ Created"
fi

echo ""
echo "✅ Environments ready"

## 4️⃣ Mount Google Drive

**IMPORTANT:** Part 1 outputs must be in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted at /content/drive/MyDrive/")

---

# ⚙️ CONFIGURATION - UPDATE THIS!

## 5️⃣ Configure Part 1 Output Path

**🔴 REQUIRED:** Update `PART1_OUTPUT_DIR` to point to your Part 1 output folder in Google Drive

---

In [ ]:
import os
import shutil

# ═══════════════════════════════════════════════════════════
# 🔴 UPDATE THIS - Point to your Part 1 output in Google Drive
# ═══════════════════════════════════════════════════════════

PART1_OUTPUT_DIR = "/content/drive/MyDrive/rtpipeline_part1_output_20250101_120000"

# The path format is usually:
# /content/drive/MyDrive/rtpipeline_part1_output_YYYYMMDD_HHMMSS
#
# Check the README_PART2.txt file from Part 1 for the exact path

# ═══════════════════════════════════════════════════════════
# Processing Options
# ═══════════════════════════════════════════════════════════

WORKERS = 4                  # CPU workers (4-8 recommended)
ENABLE_RADIOMICS = True      # Extract radiomic features
ENABLE_ROBUSTNESS = False    # Robustness testing (adds 10-30 min)

# ═══════════════════════════════════════════════════════════
# Validation and Setup
# ═══════════════════════════════════════════════════════════

# Local working directories
OUTPUT_DIR = "/content/output"
LOGS_DIR = "/content/logs"

# Check Part 1 outputs
if not os.path.exists(PART1_OUTPUT_DIR):
    print("🔴 ERROR: Part 1 output directory not found!")
    print(f"   Path: {PART1_OUTPUT_DIR}")
    print("\n   Please:")
    print("   1. Check that you completed Part 1")
    print("   2. Update PART1_OUTPUT_DIR above")
    print("   3. Check README_PART2.txt from Part 1 for correct path")
else:
    manifest_path = f"{PART1_OUTPUT_DIR}/_COURSES/manifest.json"
    if not os.path.exists(manifest_path):
        print("⚠️ WARNING: Doesn't look like a valid Part 1 output")
        print("   Missing: _COURSES/manifest.json")
    else:
        print("✅ Part 1 outputs found!")
        print(f"   {PART1_OUTPUT_DIR}")
        
        # Copy to local directory
        print("\nCopying Part 1 outputs to local directory...")
        print("(This may take several minutes)")
        
        if os.path.exists(OUTPUT_DIR):
            shutil.rmtree(OUTPUT_DIR)
        shutil.copytree(PART1_OUTPUT_DIR, OUTPUT_DIR)
        
        # Count courses
        import glob
        courses = []
        for patient_dir in glob.glob(f"{OUTPUT_DIR}/*/"):
            patient_name = os.path.basename(patient_dir.rstrip('/'))
            if patient_name.startswith('_') or patient_name.startswith('.'):
                continue
            for course_dir in glob.glob(f"{patient_dir}/*/"):
                course_name = os.path.basename(course_dir.rstrip('/'))
                if not course_name.startswith('_'):
                    courses.append(f"{patient_name}/{course_name}")
        
        print(f"\n✅ Copied! Found {len(courses)} course(s):")
        for c in courses[:5]:
            print(f"  - {c}")
        if len(courses) > 5:
            print(f"  ... and {len(courses) - 5} more")

# Create logs directory
os.makedirs(LOGS_DIR, exist_ok=True)

# Load DICOM root from Part 1 config
part1_config = f"{OUTPUT_DIR}/config_part1.yaml"
if os.path.exists(part1_config):
    with open(part1_config, 'r') as f:
        import re
        content = f.read()
        match = re.search(r'dicom_root:\s*"([^"]+)"', content)
        if match:
            DICOM_ROOT = match.group(1)
            print(f"\n✅ DICOM root from Part 1: {DICOM_ROOT}")
        else:
            DICOM_ROOT = "/content/drive/MyDrive/my_dicom_folder"
            print(f"\n⚠️ Using default DICOM root: {DICOM_ROOT}")
else:
    DICOM_ROOT = "/content/drive/MyDrive/my_dicom_folder"
    print(f"\n⚠️ No Part 1 config found, using default DICOM root")

print(f"\n📋 Configuration Summary:")
print(f"   Workers: {WORKERS}")
print(f"   Radiomics: {'✅ Enabled' if ENABLE_RADIOMICS else '❌ Disabled'}")
print(f"   Robustness: {'✅ Enabled' if ENABLE_ROBUSTNESS else '❌ Disabled'}")

## 5️⃣b Robustness Testing Configuration (Optional)

Only relevant if `ENABLE_ROBUSTNESS = True` above

In [ ]:
# Robustness testing configuration (if enabled)
ROBUSTNESS_STRUCTURES = [
    "GTV*", "CTV*", "PTV*",
    "urinary_bladder", "rectum", "prostate"
]
ROBUSTNESS_INTENSITY = "standard"  # mild, standard, aggressive

if ENABLE_ROBUSTNESS:
    print(f"Robustness Testing: {ROBUSTNESS_INTENSITY}")
    print(f"Structures: {len(ROBUSTNESS_STRUCTURES)} patterns")
else:
    print("Robustness testing disabled")

---

## 6️⃣ Generate Configuration File

In [ ]:
config_yaml = f"""# RTpipeline Configuration - Part 2 (CPU Analysis)
dicom_root: "{DICOM_ROOT}"
output_dir: "{OUTPUT_DIR}"
logs_dir: "{LOGS_DIR}"
workers: {WORKERS}

radiomics:
  params_file: "/content/rtpipeline/rtpipeline/radiomics_params.yaml"
  thread_limit: 4
  skip_rois: [body, couchsurface, bones]
  max_voxels: 1500000000
  min_voxels: 10

radiomics_robustness:
  enabled: {str(ENABLE_ROBUSTNESS).lower()}

custom_structures: "custom_structures_pelvic.yaml"
"""

config_path = "/content/config_part2.yaml"
with open(config_path, 'w') as f:
    f.write(config_yaml)

print(f"✅ Configuration written to: {config_path}")

## 7️⃣ Run Part 2 Pipeline

This runs all CPU-based analysis:
- DVH extraction
- Radiomics (if enabled)
- Robustness testing (if enabled)
- Quality control
- Results aggregation

⏱️ **Time:** 10-60 minutes depending on dataset and options

In [ ]:
import os
import subprocess

os.environ['PATH'] = f"/content/miniconda/bin:{os.environ.get('PATH', '')}"
os.chdir('/content/rtpipeline')

print("═══════════════════════════════════════════════════")
print("   RTpipeline Part 2: CPU Analysis")
print("═══════════════════════════════════════════════════")
print("\nProcessing steps (all CPU-based):")
print("  ✓ DVH extraction")
if ENABLE_RADIOMICS:
    print("  ✓ Radiomics extraction")
if ENABLE_ROBUSTNESS:
    print("  ✓ Robustness testing")
print("  ✓ Quality control")
print("  ✓ Results aggregation\n")

# Install Snakemake if needed
try:
    subprocess.run(["conda", "run", "-n", "base", "snakemake", "--version"],
                   check=True, capture_output=True)
except subprocess.CalledProcessError:
    print("Installing Snakemake...")
    subprocess.run(["conda", "install", "-n", "base", "-c", "conda-forge",
                    "-c", "bioconda", "snakemake", "-y", "-q"], check=True)
    print("✅ Snakemake installed\n")

# Run Snakemake for all steps
cmd = [
    "conda", "run", "-n", "base", "snakemake",
    "--configfile", "/content/config_part2.yaml",
    "--use-conda",
    "--cores", str(WORKERS),
    "--printshellcmds",
    "--keep-going"
]

result = subprocess.run(cmd, capture_output=False, text=True)

print("\n" + "="*50)
if result.returncode == 0:
    print("✅ Part 2 Complete!")
    print("="*50)
    print(f"\nResults: {OUTPUT_DIR}/_RESULTS/")
else:
    print("⚠️ Completed with some errors")
    print("="*50)
    print(f"\nCheck logs and results: {OUTPUT_DIR}/_RESULTS/")

## 8️⃣ View Results

Load and preview the results

In [ ]:
import pandas as pd
import os

results_dir = f"{OUTPUT_DIR}/_RESULTS"

if not os.path.exists(results_dir):
    print("⚠️ Results directory not found")
else:
    print("═══════════════════════════════════")
    print("   Results Summary")
    print("═══════════════════════════════════\n")
    
    # List files
    files = [f for f in os.listdir(results_dir) if f.endswith('.xlsx')]
    print(f"Generated {len(files)} result files:\n")
    for f in files:
        size_mb = os.path.getsize(os.path.join(results_dir, f)) / 1024 / 1024
        print(f"  ✓ {f} ({size_mb:.1f} MB)")
    
    # Load DVH
    try:
        dvh = pd.read_excel(os.path.join(results_dir, "dvh_metrics.xlsx"))
        print(f"\n📊 DVH Metrics: {len(dvh)} rows")
        print(f"   Top structures: {', '.join(dvh['Structure'].value_counts().head(5).index.tolist())}")
        globals()['dvh'] = dvh
    except Exception as e:
        print(f"\n⚠️ DVH load error: {e}")
    
    # Load radiomics
    if ENABLE_RADIOMICS:
        try:
            radiomics = pd.read_excel(os.path.join(results_dir, "radiomics_ct.xlsx"))
            print(f"\n🔬 Radiomics: {len(radiomics)} rows, {len(radiomics.columns)} features")
            globals()['radiomics'] = radiomics
        except Exception as e:
            print(f"\n⚠️ Radiomics load error: {e}")
    
    # Load robustness
    if ENABLE_ROBUSTNESS:
        try:
            rob = pd.read_excel(os.path.join(results_dir, "radiomics_robustness_summary.xlsx"),
                               sheet_name='global_summary')
            print(f"\n🎯 Robustness: {len(rob)} features analyzed")
            print(f"   Categories: {rob['robustness_label'].value_counts().to_dict()}")
            globals()['robustness_summary'] = rob
        except Exception as e:
            print(f"\n⚠️ Robustness load error: {e}")
    
    # Load metadata
    try:
        metadata = pd.read_excel(os.path.join(results_dir, "case_metadata.xlsx"))
        print(f"\n📋 Metadata: {metadata['PatientID'].nunique()} patients, {len(metadata)} courses")
    except Exception as e:
        print(f"\n⚠️ Metadata load error: {e}")
    
    print("\n✅ Results loaded into memory")
    print("   Access via: dvh, radiomics, robustness_summary, metadata")

## 9️⃣ Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

if 'dvh' in globals():
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Mean dose by structure
    top_structures = dvh.groupby('Structure')['Dmean_Gy'].mean().sort_values(ascending=False).head(10)
    top_structures.plot(kind='barh', ax=axes[0], color='steelblue')
    axes[0].set_xlabel('Mean Dose (Gy)')
    axes[0].set_title('Top 10 Structures by Mean Dose')
    
    # Volume distribution
    dvh['ROI_Volume_cc'].hist(bins=50, ax=axes[1], color='coral', edgecolor='black')
    axes[1].set_xlabel('ROI Volume (cc)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('ROI Volume Distribution')
    axes[1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()
    print("✅ DVH visualizations")
else:
    print("⚠️ No DVH data to visualize")

## 🔟 Robustness Visualizations (if enabled)

In [ ]:
if ENABLE_ROBUSTNESS and 'robustness_summary' in globals():
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # ICC distribution
    robustness_summary['icc'].hist(bins=50, ax=axes[0,0], color='steelblue', alpha=0.7)
    axes[0,0].axvline(0.90, color='green', linestyle='--', label='Robust (≥0.90)')
    axes[0,0].set_xlabel('ICC')
    axes[0,0].set_title('ICC Distribution')
    axes[0,0].legend()
    
    # CoV distribution
    robustness_summary[robustness_summary['cov_pct'] <= 100]['cov_pct'].hist(
        bins=50, ax=axes[0,1], color='coral', alpha=0.7)
    axes[0,1].axvline(10, color='green', linestyle='--', label='Robust (≤10%)')
    axes[0,1].set_xlabel('CoV (%)')
    axes[0,1].set_title('CoV Distribution')
    axes[0,1].legend()
    
    # Category distribution
    robustness_summary['robustness_label'].value_counts().plot(
        kind='bar', ax=axes[1,0], color=['green', 'orange', 'red'])
    axes[1,0].set_xlabel('Category')
    axes[1,0].set_title('Robustness Categories')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # ICC vs CoV scatter
    color_map = {'robust': 'green', 'acceptable': 'orange', 'poor': 'red'}
    for label, color in color_map.items():
        data = robustness_summary[robustness_summary['robustness_label'] == label]
        axes[1,1].scatter(data['icc'], data['cov_pct'], 
                         c=color, label=label, alpha=0.6, s=30)
    axes[1,1].axvline(0.90, color='green', linestyle='--', alpha=0.5)
    axes[1,1].axhline(10, color='green', linestyle='--', alpha=0.5)
    axes[1,1].set_xlabel('ICC')
    axes[1,1].set_ylabel('CoV (%)')
    axes[1,1].set_title('ICC vs CoV')
    axes[1,1].set_ylim([0, 100])
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*50)
    print("Robustness Summary")
    print("="*50)
    for label in ['robust', 'acceptable', 'poor']:
        count = (robustness_summary['robustness_label'] == label).sum()
        pct = 100 * count / len(robustness_summary)
        print(f"{label.capitalize()}: {count} ({pct:.1f}%)")
elif ENABLE_ROBUSTNESS:
    print("⚠️ Robustness data not loaded")
else:
    print("ℹ️ Robustness testing not enabled")

## 1️⃣1️⃣ Download Results as ZIP

In [ ]:
%%bash
echo "Creating results archive..."
cd /content
zip -r -q results.zip output/_RESULTS/
echo "✅ Archive created: /content/results.zip"
ls -lh /content/results.zip

In [ ]:
from google.colab import files

print("Downloading results.zip...")
files.download('/content/results.zip')
print("\n✅ Download started. Check your browser's downloads folder.")

## 1️⃣2️⃣ Save Results to Google Drive

In [ ]:
import shutil
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
drive_results = f"/content/drive/MyDrive/rtpipeline_results_{timestamp}"

print(f"Saving results to Google Drive...")
print(f"Destination: {drive_results}")

try:
    shutil.copytree(f"{OUTPUT_DIR}/_RESULTS", drive_results)
    
    files = [f for f in os.listdir(drive_results) if f.endswith('.xlsx')]
    
    print("\n" + "="*60)
    print("🎉 ALL DONE!")
    print("="*60)
    print(f"\nResults saved to: {drive_results}")
    print(f"\nSaved {len(files)} files:")
    for f in files:
        print(f"  ✓ {f}")
    
except Exception as e:
    print(f"\n⚠️ Error: {e}")

---

## 🎉 Pipeline Complete!

**What you have:**
- ✅ DVH metrics for all structures
- ✅ Radiomic features (if enabled)
- ✅ Robustness analysis (if enabled)
- ✅ Quality control reports
- ✅ Analysis-ready Excel files

**💰 Cost Savings:**
By splitting GPU (Part 1) and CPU (Part 2), you've optimized your Colab costs!

---

## 📚 Resources

- **Output Format Guide:** [output_format.md](https://github.com/kstawiski/rtpipeline/blob/main/output_format.md)
- **Robustness Guide:** [RADIOMICS_ROBUSTNESS.md](https://github.com/kstawiski/rtpipeline/blob/main/RADIOMICS_ROBUSTNESS.md)
- **Repository:** https://github.com/kstawiski/rtpipeline

---

**Notebook Version:** 2.0 (Part 2 - CPU Analysis)  
**Compatible with:** rtpipeline v2.0+ as of November 2025